In [13]:
import pandas as pd
# se combinan datos de metereologia y particulas contaminantes y se calcula icap en base a pm10

In [14]:
d_pm25 = pd.read_csv('../data/processed/pm_2.5.csv',sep=';', thousands=None, decimal=',')
d_pm10 = pd.read_csv('../data/processed/pm_10.csv',sep=';', thousands=None, decimal=',')

Metereologia = pd.read_csv('../data/processed/Metereologia.csv',sep=';', thousands=None, decimal=',')


In [15]:
d_contaminantes = pd.merge(d_pm25, d_pm10, on='fecha', how='outer')

nuevos_nombres = {
    'registro_x': 'pm2.5',
    'registro_y': 'pm10'
}
d_contaminantes = d_contaminantes.rename(columns=nuevos_nombres)
# Eliminando la fila con índice 0
d_contaminantes.drop(index=0, inplace=True)
d_contaminantes.reset_index(drop=True, inplace=True)

In [16]:
Metereologia.rename(columns={'ff_Valor': 'viento'}, inplace=True)
Metereologia.rename(columns={'dd_Valor': 'dir_viento'}, inplace=True)

DataSet = pd.merge(d_contaminantes, Metereologia, on='fecha', how='outer')

In [17]:
# ETIQUETACION DE CALIDAD DEL AIRE BASADO EN ICAP
# ICAP            MP10 µg/m3N (24 horas)
# 0                0
# 100              150
# 500              330


def ICAP(PM10):

    if PM10 < 0:
        raise ValueError("PM10 debe ser mayor igual a 0")
    

    if 0 <= PM10 <= 150:
        ICAP = PM10 / 1.5
    elif 150 <PM10 :
        ICAP = (PM10 - 114) / 0.36

    
    return ICAP

DataSet['icap'] = DataSet['pm10'].apply(ICAP)

print(DataSet)

           fecha    pm2.5     pm10  humedad  puntoRocio  temperatura  \
0     2013-01-01   9.3612  25.2396  44.8462     10.0615      25.0308   
1     2013-01-02  13.7865  36.1072  43.3077     12.1000      25.8000   
2     2013-01-03  12.2568  29.2324  62.8462     13.4000      21.2462   
3     2013-01-04  12.6945  33.1919  68.6923     13.5385      19.6000   
4     2013-01-05   8.6662  26.3441  53.5385     12.3231      23.3231   
...          ...      ...      ...      ...         ...          ...   
3737  2023-03-27  18.5833  47.5000  53.8750      5.0792      15.8042   
3738  2023-03-28   5.8125  23.0000  78.2500      8.4962      12.2885   
3739  2023-03-29   5.3542  21.5000  63.3750      5.4625      13.4583   
3740  2023-03-30  10.3333  32.0000  55.0417      2.6625      12.6625   
3741  2023-03-31  20.8333  47.0000  54.0000      3.7625      15.5917   

      dir_viento  viento       icap  
0       174.6154  3.7692  16.826400  
1       256.1538  3.7692  24.071467  
2       213.0769  2.7

In [18]:
nombre = 'Dataset_Particulas_contaminantes'

path = '../data/processed/{}.csv'.format(nombre)


DataSet.to_csv(path, sep=';',index=False, float_format='%.4f', decimal=',')